In [1]:
import numpy as np
import collections 
import random as rd


In [155]:
class puzzle:
    NORTH ,SOUTH,EAST ,WEST = 1,2,3,4
    board = None
    parent = None
    depth = None
    neighbours = None
    
    def __init__(self,board,parent=None,depth=0):
        self.board = board
        self.parent = parent
        self.depth = depth
    
    def getMoves(self):
        
        return
    
    #Funcion que realiza los merges de los numeros iguales
    def mergeNumbers(self,obj,dest):
        
        if self.board[obj] == self.board[dest]:
            self.board[dest] = self.board[obj] + self.board[dest]
            self.board[obj] = 0
            return True
        return False
            
    
    
    #Funcion que realiza los movimientos de las fichas
    def moveBoard(self,move):
        board = self.board.copy()
        moves = []
        
        
        if move == self.NORTH:
            #Recorremos las fichas de arriba hacia abajo
            for i in range(1,4):
                for j in range(4):
                    moves.append([(i,j),(i-1,j)])
        
        if move == self.SOUTH:
            #Recorremos las fichas de abajo hacia arriba
            for i in reversed(range(0,3)):
                for j in range(3):
                    moves.append([(i,j),(i+1,j)])
            
        if move == self.EAST:
            #Recorremos las fichas de derecha a izquierda
            for j in reversed(range(0,3)):
                for i in range(4):
                    moves.append([(i,j),(i,j+1)])
                    
        if move == self.WEST:
            #Recorremos las fichas de izquierda a derecha
            for j in range(1,3):
                for i in range(4):
                    moves.append([(i,j),(i,j-1)])
        
        print moves
        
        
        
        
        
    

In [159]:
def CreaEstado():
    board = np.zeros((4,4))
    possvalues = [2,4]
    for i in range(2):
        randpos = (rd.randint(0,3),rd.randint(0,3))
        randvalue = rd.randint(0,1)
        if board[randpos] == 0:
            board[randpos] = possvalues[randvalue]
        else:
            i-=1
    
    return puzzle(board)

startObj = CreaEstado()
print startObj.board
print startObj.moveBoard(1)

[[0. 0. 4. 0.]
 [0. 0. 2. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[(1, 0), (0, 0)], [(1, 1), (0, 1)], [(1, 2), (0, 2)], [(1, 3), (0, 3)], [(2, 0), (1, 0)], [(2, 1), (1, 1)], [(2, 2), (1, 2)], [(2, 3), (1, 3)], [(3, 0), (2, 0)], [(3, 1), (2, 1)], [(3, 2), (2, 2)], [(3, 3), (2, 3)]]
None
